In [ ]:
!pip install bpemb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


GPT generates all the words sequentially  (generatively pre-trained transformer)
GPT is a probabilistic system for any one promt it will give us multiple answers
Lnguage model: it models sequence of words or characters or tokens(in general) it knows how words follow eachother in english language....what GPT is doing is it is completeling the sequence ...you give the start of the sequence it will complete the sequencence .

What neural netwrok our GPT-2 is using to model the sequence of words? .... attention is all you need research paper(2017)


we wil build character level language model using transformer architecture....in this model we will get character by character(our model predicts future character) ....but in real chatgpt you will get in token by token (predicts future token.....tokens are not necessarily words they can be sub-words too....like "wor","word","elephant","ele", "ant")

In [ ]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
from bpemb import BPEmb

In [ ]:
batch_size=64
block_size=256
max_iters= 5000
eval_interval =500
learning_rate =3e-4

device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters =200
n_emdb=384
n_head=6
n_layer = 6
dropout=0.2


In [ ]:
torch.manual_seed(1242)

In [ ]:
folder_path = "/content/drive/MyDrive/dickens"   #path to the folder where you dicken dataset is present

In [ ]:
#lets create a list of all text files present inside the dicken floder
text_files_list = []
for file_name in os.listdir(folder_path):    #os.listdir(folder_path) retrives all the file names inside the folderpath
  if file_name.endswith(".txt"):             #get all the files that has .txt extention
    text_files_list.append(os.path.join(folder_path,file_name))     #append all the file paths to the empty list we created earlier



In [ ]:
print(text_files_list) #you got all the file paths in a list

['/content/drive/MyDrive/dickens/1467-0.txt', '/content/drive/MyDrive/dickens/1289-0.txt', '/content/drive/MyDrive/dickens/564-0.txt', '/content/drive/MyDrive/dickens/27924-0.txt', '/content/drive/MyDrive/dickens/650-0.txt', '/content/drive/MyDrive/dickens/644-0.txt', '/content/drive/MyDrive/dickens/653-0.txt', '/content/drive/MyDrive/dickens/1400-0.txt', '/content/drive/MyDrive/dickens/580-0.txt', '/content/drive/MyDrive/dickens/678-0.txt', '/content/drive/MyDrive/dickens/766-0.txt', '/content/drive/MyDrive/dickens/807-0.txt', '/content/drive/MyDrive/dickens/700-0.txt', '/content/drive/MyDrive/dickens/786-0.txt', '/content/drive/MyDrive/dickens/882-0.txt', '/content/drive/MyDrive/dickens/675-0.txt', '/content/drive/MyDrive/dickens/963-0.txt', '/content/drive/MyDrive/dickens/914-0.txt', '/content/drive/MyDrive/dickens/924-0.txt', '/content/drive/MyDrive/dickens/883-0.txt', '/content/drive/MyDrive/dickens/917-0.txt', '/content/drive/MyDrive/dickens/967-0.txt', '/content/drive/MyDrive/di

In [ ]:
# Function to combine text files
def combine_text_files(file_paths):
    with open("combined_texts.txt", "w", encoding="utf-8") as combined_file:
        for file_path in file_paths:
            # Get the file name without the directory path
            title = os.path.basename(file_path)
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
                # Write title (file name) before content
                combined_file.write(f"Title: {title}\n")
                combined_file.write(content + "\n\n")  # Separate content with newlines

# Call the function to combine the files
combine_text_files(text_files_list)

In [ ]:
with open("combined_texts.txt","r",encoding="utf-8") as f:
  text = f.read()

In [ ]:
print("text length inside the dataset",len(text))

text length inside the dataset 24351412


In [ ]:
print(text[2000:3000])

hed
tables, chairs, bedsteads, wardrobes, eight-day clocks, and various other
articles of domestic furniture (wonderfully made, in tin, at
Wolverhampton), perched among the boughs, as if in preparation for some
fairy housekeeping; there were jolly, broad-faced little men, much more
agreeable in appearance than many real men—and no wonder, for their heads
took off, and showed them to be full of sugar-plums; there were fiddles
and drums; there were tambourines, books, work-boxes, paint-boxes,
sweetmeat-boxes, peep-show boxes, and all kinds of boxes; there were
trinkets for the elder girls, far brighter than any grown-up gold and
jewels; there were baskets and pincushions in all devices; there were
guns, swords, and banners; there were witches standing in enchanted rings
of pasteboard, to tell fortunes; there were teetotums, humming-tops,
needle-cases, pen-wipers, smelling-bottles, conversation-cards,
bouquet-holders; real fruit, made artificially dazzling with gold leaf;
imitation apples

vocabulary size: we will take all non repeated characters into this vocabulary


In [ ]:
bpem_en = BPEmb(lang="en")

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:00<00:00, 545351.78B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:01<00:00, 3620498.91B/s]


In [ ]:
vocab_size,bpem_embed_size = bpem_en.vectors.shape
print(f"vocab_size : {vocab_size}")
print(f"bpem_embed_size : {bpem_embed_size}")

vocab_size : 10000
bpem_embed_size : 100


In [ ]:
import torch
from tqdm import tqdm
import numpy as np

In [ ]:
chars = bpem_en.encode(text)

In [ ]:
token_seq=np.array(bpem_en.encode_ids(text))

In [ ]:
data = torch.tensor(token_seq,dtype = torch.long)

In [ ]:
print(data.shape)
print(data[:20]) #all 20 tokens(chars) are wrapped in a tensor

torch.Size([7068560])
tensor([1676, 9948,   52, 1236, 9935, 9915,  912,    0,  228, 1361,   51,   84,
        8650,   57, 3678, 9934,  424, 5071, 3348, 9934])


In [ ]:
# chars = sorted(list(set(text)))
# vocab_size = len(sorted(list(set(text))))
# print("".join(chars))
# print(vocab_size)

In [ ]:
# from transformers import GPT2Tokenizer

# # Read the text data from the file or any source
# with open("combined_texts.txt", "r", encoding="utf-8") as f:
#     text = f.read()

# # Limit the text to the first 10,000 characters
# text_subset = text[:10000]

# # Create a GPT-2 tokenizer and build vocabulary from the subset
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# # Tokenize the subset of text and build the vocabulary
# tokens = tokenizer(text_subset, add_special_tokens=False)['input_ids']
# custom_vocab_words = [tokenizer.decode(token) for token in tokens]

# # Display the created vocabulary
# print("Custom Vocabulary (First 50 tokens):")
# print(custom_vocab_words[:50])



lets try and tokenize the input text
this is the simplest form of encoding there are many other ways to do this too...

sentence piece will also encode text into integers but in different way they use sub-words (it has different vocabulary too)

tiktoken ---uses bipair encoding tokeniser (gpt-2 uses this tokenising)...it has 50257 vocab size ...so when we input "hii akash" string it might be tokenised to only 2 or 3 integers ...if you are using a big vocab_size your encoding size will be reduced but if you are using small vocab size like (120 in out case) encoding seq chars length will be more

In [ ]:
# string_to_integer = {ch:i for i,ch in enumerate(chars)}
# integer_to_string = {i:ch for i,ch in enumerate(chars)}
# encode = lambda s: [string_to_integer[c] for c in s]   #takes the string "hii akash" and converts that into intergers
# decode = lambda l: "".join(integer_to_string[i] for i in l)

# print(encode("hii akash"))
# print(decode(encode("hii akash")))

In [ ]:
# pip install transformers


if you want to test the how tokens work in GPT-2 uncommnet the below code

In [ ]:
# from transformers import GPT2Tokenizer

# # Load GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# def encode(text):
#     # Tokenize the text using GPT-2 tokenizer
#     tokens = tokenizer.encode(text, add_special_tokens=False)  # Set add_special_tokens=False to exclude special tokens like [CLS] and [SEP]
#     return tokens

# def decode(tokens):
#     # Decode the tokens back into text
#     text = tokenizer.decode(tokens)
#     return text

# # Example text
# input_text = "hii akash"

# # Encode the text into GPT-2 tokens
# encoded_tokens = encode(input_text)
# print("Encoded tokens:", encoded_tokens)

# # Decode the tokens back into text
# decoded_text = decode(encoded_tokens)
# print("Decoded text:", decoded_text)


lets tokenise our entire text and wrap it in a tensor

In [ ]:
# data = torch.tensor(encode(text),dtype = torch.long)
# print(data.shape)
# print(data[:20]) #all 20 tokens(chars) are wrapped in a tensor

we will not feed entire text into the transformer we will feed chunks of data while training (max sequence length is 8 here in our model)

GPT-2 has a block size of 1024 (max sequence length)

In [ ]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [ ]:
# block_size = 8  # chuck size we feed into transformer
train_data[:block_size+1]

tensor([1676, 9948,   52, 1236, 9935, 9915,  912,    0,  228, 1361,   51,   84,
        8650,   57, 3678, 9934,  424, 5071, 3348, 9934,  101, 1976, 5993,  304,
           0, 4929,   57, 3678,   80,   72,    7,  766,   27, 7822,  756, 3482,
          26,    7,  593,  590,   34,  434,    0,  730, 2277,   27,    7,  501,
         114,  467, 1829,   34,   97, 2308,  467, 9647, 9912,    0, 3863,  217,
         649, 9935, 1220,  437, 5169,  107, 9934, 3090,  107, 2318,  127,   50,
        9940, 2987,  107,  400,    7, 2649,   27,    0,  228, 1361,   51,   84,
        8650, 7044, 1364,   97,  215,   57, 3678,  127, 3663,  114, 9912,    0,
        9931, 9931, 9931, 9935, 9930,   84, 8650, 9935, 6911, 9935,  875, 1220,
         154,  216, 1251,   26,    7,  593,  590, 9934, 1220, 9937, 1188,  309,
           0,  846, 5952,    7, 3319,   27,    7, 1322,  571, 1220,  154, 1251,
         745, 1359,  215,   57, 3678, 9935,    0, 9915,   25,   54, 9948,  424,
        5071, 3348,    0, 1896, 1067, 99

while training we train like input 52 output 70
when input is 52,70 then output is 81
when input is 52,70,81 then ouptput is 73
....we train all the way till we reach max sequence length(8 in our case)

piupose of this is to make transformer see all the way from start to end
we are continuously teaching the transformer the context between the letters

so even if you enter 1 letter transformer will give something

In [ ]:
x=train_data[:block_size]
y=train_data[1:block_size+1] #y is the prediction we want to make say x= 52,70 then y is 81 and if x is 52,70,81 then y is 73
# print(x)
# print(y)
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"if the input is {context} then output is {target}")

if the input is tensor([1676]) then output is 9948
if the input is tensor([1676, 9948]) then output is 52
if the input is tensor([1676, 9948,   52]) then output is 1236
if the input is tensor([1676, 9948,   52, 1236]) then output is 9935
if the input is tensor([1676, 9948,   52, 1236, 9935]) then output is 9915
if the input is tensor([1676, 9948,   52, 1236, 9935, 9915]) then output is 912
if the input is tensor([1676, 9948,   52, 1236, 9935, 9915,  912]) then output is 0
if the input is tensor([1676, 9948,   52, 1236, 9935, 9915,  912,    0]) then output is 228
if the input is tensor([1676, 9948,   52, 1236, 9935, 9915,  912,    0,  228]) then output is 1361
if the input is tensor([1676, 9948,   52, 1236, 9935, 9915,  912,    0,  228, 1361]) then output is 51
if the input is tensor([1676, 9948,   52, 1236, 9935, 9915,  912,    0,  228, 1361,   51]) then output is 84
if the input is tensor([1676, 9948,   52, 1236, 9935, 9915,  912,    0,  228, 1361,   51,   84]) then output is 8650
if 

we will feed multiple chuncks stacked uopn one another (batch_size) to transformer...this is to keep GPU busy

In [ ]:
# torch.manual_seed(43)
# ix = torch.randint(len(data)-block_size,(4,))
# print(ix)

# z=[data[i:i+block_size] for i in ix] # ----->outputs a list of 4 tensors and we stack them one on top of another
# print(z)

# x=torch.stack([data[i:i+block_size] for i in ix])
# y=torch.stack([data[i+1:i+block_size+1] for i in ix])
# print(x)
# print(y)


xb is something that we feed into transformer and yb is targets that we use at the end while calculating loss

In [ ]:
# torch.manual_seed(43)
# block_size=8        #max_sequence length
# batch_size =4   #batch size


def batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  # print(ix)
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device),y.to(device)                              #while loading the data we have to move it to device
  return x,y


@torch.no_grad()         #everythng that is happening inside we will not call .Backward() function...it does not have to store all the intermediate varibales ..so memory use is efficient
def estimate_loss():
  out={}
  model.eval()
  for split in ["train","val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = batch("split")
      logits,loss= model(X,Y)
      losses[k]=loss.item()
    out[split]=loss.mean()
  model.train()
  return out


# xb,yb = batch(train_data)
# print(f"xb is {xb}")
# print(f"yb is {yb}")

# for b in range(batch_size):
#   for t in range(block_size):
#     context = xb[b,:t+1]
#     target = yb[b,t]
#     print(f"when input is {context.tolist()} the target is  {target}")


In [ ]:
# print(xb)

The nn.Embedding layer is a simple lookup table that maps an index value to a weight matrix of a certain dimension. the vocabulary size and the size of the encoded representation for each word.

 every single integer in the input(xb) will refer to the embedding table and will pluck out the row in the embedidng table corresponding to its index

 each interger in xb has 120 size vector representation...1st

  [[[120],[120],[120],[120],[120],[120],[120],[120]],

   [[120],[120],[120],[120],[120],[120],[120],[120]],

   [[120],[120],[120],[120],[120],[120],[120],[120]],

   [[120],[120],[120],[120],[120],[120],[120],[120]]]



   you can also observe that each token has an embedding vector that is why the size is (vocab_size,embedding_size)

In [ ]:
#embedding layer
c=torch.randn((27,2))  #this is a simple lookup table with 27 rows and 2columns each input is represented in vector of size 2
# print(c)
print(c[5])

tensor([ 0.8141, -1.5440])


logits are the scores for the next character in the sequence

we have multidimentional input (B,T,C) But, while calculating loss it expects input channel to be second i.e, (B,C,T) instead of (B,T,C)

In [ ]:
torch.manual_seed(1337)


class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.key= nn.Linear(n_emdb,head_size,bias=False)
    self.query= nn.Linear(n_emdb,head_size,bias=False)
    self.value= nn.Linear(n_emdb,head_size,bias=False)
    self.register_buffer("tril",torch.tril(torch.ones(block_size,block_size))) #tril is not a parameter it is a buffer

    self.dropout=nn.Dropout(dropout)


  def forward(self,x):
    B,T,C=x.shape
    k=self.key(x)
    q=self.query(x)
    wei = q@k.transpose(-2,-1)*C**-0.5
    wei= wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
    wei =F.softmax(wei,dim=-1)
    wei=self.dropout(wei)
    v=self.value(x)
    out = wei @ v
    return out


class MultiHeadAttention(nn.Module):

  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_emdb,n_emdb)
    self.dropout=nn.Dropout(dropout)

  def forward(self,x):
    out= torch.cat([h(x) for h in self.heads], dim=-1) #concate in channel dimenssion
    out=self.dropout(self.proj(out))
    # out=self.proj(out)
    return out


class FeedForward(nn.Module):
  def __init__(self,n_emdb):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_emdb,4*n_emdb),
        nn.ReLU(),
        nn.Linear(4*n_emdb,n_emdb),
        nn.Dropout(dropout),
    )

  def forward(self,x):
    return self.net(x)


class Block(nn.Module):
  def __init__(self,n_emdb,n_head):
    super().__init__()
    head_size=n_emdb//n_head
    self.sa= MultiHeadAttention(n_head,head_size)
    self.ffwd = FeedForward(n_emdb)
    self.ln1=nn.LayerNorm(n_emdb)            #layer norm
    self.ln2=nn.LayerNorm(n_emdb)


  def forward(self,x):
    x=x+self.sa(self.ln1(x))                      #before x enters self attention and feed frwd apply layer norm
    x=x+self.ffwd(self.ln2(x))
    return x



class Bingram_model(nn.Module):
  def __init__(self):          #vocab_size is called my instance of class is created
    super().__init__()   #calls the constructor of the parent class (nn.Module). This line ensures that necessary initializations defined in the parent class are also executed.
    self.token_embedding_table = nn.Embedding(vocab_size,n_emdb)   #(120,120)
    self.position_embedding_table = nn.Embedding(block_size,n_emdb) #each position will get embedding vector
    self.blocks = nn.Sequential(*[Block(n_emdb,n_head=n_head) for _ in range(n_layer)])    #number of  layers of blocks
    self.ln = nn.LayerNorm(n_emdb)
    # self.sa_head= Head(n_emdb)
    # self.blocks = nn.Sequential(
    #     Block(n_emdb,n_head=4),
    #     Block(n_emdb,n_head=4),
    #     Block( n_emdb,n_head=4),
    #     nn.LayerNorm(n_emdb)
    # )

    # self.sa_heads = MultiHeadAttention(4,n_emdb//4)
    # self.ffwd= FeedForward(n_emdb)
    self.lm_head=nn.Linear(n_emdb,vocab_size)

  def forward(self,idx,targets=None):        #None is given because in generate function we are no having targets if you don't give None it will give error
    #idx shape is (B,T) and targets are also (B,T)   (4,8)
    # print(idx)
    # print(self.token_embedding_table)
    # print(self.token_embedding_table.shape)
    B,T = idx.shape
    tok_emb= self.token_embedding_table(idx)     #(B,T,C=n_embd)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) #(T,C)...0 to T elemnets are embedded
    x=tok_emb + pos_emb  #(B,T,C)
    # x=self.sa_heads(x)
    # x=self.ffwd(x)
    x=self.blocks(x)
    logits=self.lm_head(x) #(B,T,C=vocab_size)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits=logits.view(B*T,C)    # WE are taking all integer positions inputs(81,66...all the elements in the rows and columns into rows) and 120 emebdding vector as column
      # print(logits.shape)
      # print(logits)
      targets = targets.view(B*T) #also can use targets.view(-1)
      loss =  F.cross_entropy(logits,targets)
      # print(logits[0])
      # print(logits)
      # print(logits.shape)

    return logits,loss

  def generate(self, idx, max_new_tokens):
      # idx is (B, T) array of indices in the current context
      for _ in range(max_new_tokens):
          idx_cond = idx[:,-block_size:]
          # get the predictions
          logits, loss = self(idx_cond)
          # focus only on the last time step
          logits = logits[:, -1, :] # becomes (B, C)
          # apply softmax to get probabilities
          probs = F.softmax(logits, dim=-1) # (B, C)
          # sample from the distribution
          idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
          # append sampled index to the running sequence
          idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
      return idx




when you move the model to device all the parameters like embedding weights all of them will run on gpu and will work much faster than cpu

In [ ]:
model = Bingram_model()
m=model.to(device)        #when we are creating the model move the model parameters to the device
# logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))
# print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

lets create a pytorch optimizer

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-3)

let us use gradients and update parameters

In [ ]:
from tqdm import tqdm

In [ ]:
# batch_size=32

for iter in tqdm(range(max_iters)):
  #once in a while find train loss and val loss
  if iter % eval_interval ==0:
    losses = estimate_loss()                #estimate loss averages the loss over multiple batches
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
  xb,yb = batch("train")
  logits,loss = model(xb,yb)       #feeding batches into the model and getting loss and logits
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# print(loss.item())

  0%|          | 0/5000 [00:00<?, ?it/s]

step 0: train loss 9.6682, val loss 9.6647


 10%|█         | 500/5000 [06:13<45:40,  1.64it/s]

step 500: train loss 4.6145, val loss 4.6714


 20%|██        | 1000/5000 [12:49<41:04,  1.62it/s]

...

step 2000: train loss 3.8506, val loss 3.9763


 50%|█████     | 2500/5000 [32:50<25:44,  1.62it/s]

step 2500: train loss 4.0056, val loss 3.9862


 50%|█████     | 2523/5000 [34:33<33:55,  1.22it/s]


KeyboardInterrupt: ignored

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long,device = device)

In [ ]:
print(bpem_en.decode(m.generate(context, max_new_tokens=500)[0].tolist()))


In [ ]:
torch.save(model.state_dict(), 'trained_model.pth')

In [ ]:
# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

In [ ]:
# # Assuming 'model' is your trained model
# def answer_question(question):
#     # Preprocess the question (tokenization, padding, etc.) as required by your model

#     # Convert the question to model input format
#     processed_question = preprocess_question(question)

#     # Assuming 'model' is a PyTorch model
#     with torch.no_grad():
#         logits = model(processed_question)
#         # Perform any necessary post-processing on the logits to get the answer
#         answer = postprocess_logits(logits)

#     return answer

# # Example usage
# user_question = input("Enter your question: ")
# answer = answer_question(user_question)
# print("Answer:", answer)


In [ ]:
from google.colab import files

# Replace 'file_path' with the path to your file in Colab
files.download('/content/trained_model.pth')


self attention block: gathers the nformation from the past in a datadependent way

communicaion mechanism between the nodes



8 tokens hey are not talking to eah other we would like to them to communicate eachother. but token at 5th position should only communicaae with the 1,2,3,4 tokens but not 6th,7th,8th(becuse these are the token we want to predict)  

how to communicate? average of all preceeding elements
example 5th token take channels(that 120 vector) at 5th+4th+3rs+2nd+1st ..... result will give 5th token w.r.t to the context of 1,2,3,4,5 (this is  a weak communication since it is just an average...because we have lost so much information about the speial location of the tokens)



suppose token1 is [0.2,0.3,0.5]
token 2 is [0.1,0.1,0.5]....after creating context 2nd token will become [0.15,0.2,0.5]


initially we will give weights as 0 ..later these weights will get updated ...we can think these weights are the interactioin strength




every single token at each position will emit two vectors it wll emit a query and it will emit a key ..query vector --what am i looking for ...and key --what do i contain  ................we get affinity by doing dot product key and query....if key and query are self aligned they will produce high amount


k = key(x)...ALL the tokens in all the positions in (B,T) arrangement all of them in parallel independently produces a key and query ... no communication happened yet ....when dot product happened communication happens


wei each token knows where its position is and what is the context in it


we divide by sqrt(head_size) to preserve variance ..if you don't do this step the softmax function may converge to one-hot encoder if there are any very large positive values  or very large negative values (we want values to be diffused when we are initializing)

In [ ]:
# torch.manual_seed(1337)
# B,T,C = 4,8,32
# x=torch.randn(B,T,C)
# # print(x.shape)
# head_size = 16
# key=nn.Linear(C,head_size,bias=False) #Linear(in_features=32, out_features=16, bias=False)
# # print(key)
# query=nn.Linear(C,head_size,bias=False)
# value=nn.Linear(C,head_size,bias=False)
# k=key(x)  #[B,T,16]
# # print(k.shape)
# q=query(x)  ##[B,T,16]

# wei = q @ k.transpose(-2,-1) #transpose only T,C not B
# #(B,T,16) @ (B,16,T)=(B,T,T)
# print(wei[0])

# tril =torch.tril(torch.ones(T,T))
# # print(tril)
# # wei = torch.zeros((T,T))
# # print(wei)
# wei = wei.masked_fill(tril==0,float('-inf'))
# # print(wei)
# wei= F.softmax(wei,dim=-1) #we will get nice distribution without any negative values
# # print(wei)

# v= value(x) #what i have what i can communicate
# # out = wei @ x  #(B,T,T) @ (B,T,C) = (B,T,C)
# out = wei @ v
# # print(out)
# out.shape

attention acts over set of vectors and these nodes have no idea where they are positioned in space  that is we encode them positionally

In [ ]:
# wei[0]
# #

In [ ]:
# torch.manual_seed(42)
# a=torch.tril(torch.ones(3,3))  #torch.tril give lower triangular matrix
# print(a)

In [ ]:
# wei=wei.masked_fill(tril==0,float('-inf')) #tril uppet part is 0's noow -inf will replace these zeros
#apply softmax it is like normalization

In [ ]:
# torch.softmax(torch.tensor([0.1,0.2,0.15,5]),dim=-1)

feed dorward is simple multulayer perceptron..after  multihead attention directly passed it to get logits but if you do this tokens will look at each other but they will not have time to think what they have found in other tokens  .....multihead attention is for communication and feedfrwd is for computation

After introducing blocks you will notice results will get bad ..this is because we are using deep layers .so there are 2 optimization that will dramatically help with the deep neural networks 1) skip connections or residual connections
